In [1]:
# https://stackoverflow.com/a/67029719/7782
from IPython import get_ipython
from IPython.core.magic import register_cell_magic

ipython = get_ipython()

@register_cell_magic
def pybash(line, cell):
    ipython.run_cell_magic('bash', '', cell.format(**globals()))

In [2]:
import argparse
import os
import types
import datetime


import ezid_client_tools as ect
from ezid_client_tools.utils import ANVL

import structured_ezid as sezid

import settings



In [3]:
EZID_USER = os.environ.get('EZID_USER')
EZID_PASSWD = os.environ.get('EZID_PASSWD')

if (EZID_USER is None) or (EZID_PASSWD is None):
    import settings
    EZID_USER = settings.EZID_USER
    EZID_PASSWD = settings.EZID_PASSWD

TEST_NAAN = "99999"
TEST_SHOULDER = "fk4"
TEST_ID = "isamplestest"
TEST_PROJECT_ID = "prefixmatch"


# New shoulder

OC_NAAN = "28722"
OC_PREREG_SHOULDER = "r2"

In [11]:
client2.ezid_url

'https://ezid-stg.cdlib.org'

In [20]:
client2 = sezid.Client2()
client2.args.credentials = f"{EZID_USER}:{EZID_PASSWD}"
client2.args.server = "s"

arks_to_create = (
  sezid.ARKIdentifier(TEST_NAAN, TEST_SHOULDER, f"{TEST_ID}"),
  sezid.ARKIdentifier(TEST_NAAN, TEST_SHOULDER, f"{TEST_ID}/{TEST_PROJECT_ID}"),
  sezid.ARKIdentifier(TEST_NAAN, TEST_SHOULDER, f"{TEST_ID}/{TEST_PROJECT_ID}/a"),
  sezid.ARKIdentifier(TEST_NAAN, TEST_SHOULDER, f"{TEST_ID}/{TEST_PROJECT_ID}/a/b"),
  sezid.ARKIdentifier(TEST_NAAN, TEST_SHOULDER, f"{TEST_ID}/{TEST_PROJECT_ID}/a/c"),
  sezid.ARKIdentifier(TEST_NAAN, TEST_SHOULDER, f"{TEST_ID}/{TEST_PROJECT_ID}/a/c1"),
  sezid.ARKIdentifier(TEST_NAAN, TEST_SHOULDER, f"{TEST_ID}/{TEST_PROJECT_ID}/a/c1/d"),
)

In [21]:
for ark_ in arks_to_create:
    
    dt = datetime.datetime.utcnow()

    metadata_ = {
      "profile": "erc",
      "erc.who": "Raymond Yee",
      "erc.what": ark_.postfix,
      "erc.when": dt.replace(microsecond=0).isoformat(),
    }

    try:
        (response, headers, status) = client2.create_identifier(ark_, metadata_, update=True)
    except ect.ClientError as e:
        if isinstance(e, ect.HTTPClientError):
            print (e.status, str(e))
        else:
            print(e, type(e))
    else:
        print(response, status)
    

    

success: ark:/99999/fk4isamplestest 201
success: ark:/99999/fk4isamplestest/prefixmatch 201
success: ark:/99999/fk4isamplestest/prefixmatch/a 201
success: ark:/99999/fk4isamplestest/prefixmatch/a/b 201
success: ark:/99999/fk4isamplestest/prefixmatch/a/c 201
success: ark:/99999/fk4isamplestest/prefixmatch/a/c1 201
success: ark:/99999/fk4isamplestest/prefixmatch/a/c1/d 201


In [26]:
# ark:/99999/fk4isamplestest/prefixmatch/a/c1/d -> a/c1/d
# ark:/99999/fk4isamplestest/prefixmatch/a/c/e -> a/c 
# ark:/99999/fk4isamplestest/prefixmatch/a/c1/e -> a/c1
# ark:/99999/fk4isamplestest/prefixmatch/a/c12/d/e -> /a (not /a/c1)

ark_ = sezid.ARKIdentifier(s='ark:/99999/fk4isamplestest/prefixmatch/a/c12/d/e')


(r, r1, headers, status) = client2.view_identifier(ark_, prefix_matching=True)
r1

LastUpdatedOrderedDict([('success',
                         'ark:/99999/fk4isamplestest/prefixmatch/a/c1 in_lieu_of ark:/99999/fk4isamplestest/prefixmatch/a/c12/d/e'),
                        ('erc.who', 'Raymond Yee'),
                        ('profile', 'erc'),
                        ('erc.what', 'isamplestest/prefixmatch/a/c1'),
                        ('erc.when', '2023-05-26T16:58:56'),
                        ('_owner', 'opencontext'),
                        ('_ownergroup', 'ucblibrary'),
                        ('_created', '1685120006'),
                        ('_updated', '1685120337'),
                        ('_profile', 'erc'),
                        ('_target',
                         'https://ezid-stg.cdlib.org/id/ark:/99999/fk4isamplestest/prefixmatch/a/c1'),
                        ('_status', 'public'),
                        ('_export', 'yes')])

In [ ]:
headers.items()

In [28]:
class Client3(sezid.Client2):
    pass

In [ ]:
# check on prefix matching

try:
    client2.view_identifier(str(ark_) + "/1234", prefix_matching=False)
except ect.ClientError as e:
    print (e)
    print (type(e), str(e))

```
ark:/99999/fk4isamplestest
ark:/99999/fk4isamplestest/prefixmatch
ark:/99999/fk4isamplestest/prefixmatch/a
ark:/99999/fk4isamplestest/prefixmatch/a/b
ark:/99999/fk4isamplestest/prefixmatch/a/c
ark:/99999/fk4isamplestest/prefixmatch/a/c1
ark:/99999/fk4isamplestest/prefixmatch/a/c1/d

ark:/99999/fk4isamplestest/prefixmatch/a/c1/d -> a/c1/d
ark:/99999/fk4isamplestest/prefixmatch/a/c/e -> a/c (not a/c1)
ark:/99999/fk4isamplestest/prefixmatch/a/c1/e -> a/c1
ark:/99999/fk4isamplestest/prefixmatch/a/c12/d/e -> /a
```


In [ ]:
# practice entering metadata at the various parts along the chain

n = 30

metadata_ = {
  "profile": "erc",
  "erc.who": "Raymond Yee",
  "erc.what": "long testing ark with clusters of 999 and 30 levels",
  "erc.when": "(:unkn) Unknown time"
}

postfix = "/".join(('999',) * n)

ark_ = sezid.ARKIdentifier(TEST_NAAN, TEST_SHOULDER, postfix)

client2.create_identifier(ark_, metadata_, update=True)


In [ ]:
p = pathlib.PurePath(ark_.postfix)
p.name, (p.stem, p.suffix)

# check whether there is metadata directly for ark_. If not, then do prefix lookup for the parent -- maybe need to do a recursive look up the tree until we hit an actual ancestor.

p.parent

In [ ]:
from pathlib import PurePath as P
P('a/b/c/d1/e'), P('a/b/c/d'), P('a/b/c/d1')

P('a/b/c/d1/e').is_relative_to(P('a/b/c/d1')), P('a/b/c/d1/e').is_relative_to(P('a/b/c/d'))

In [ ]:
client2.view_identifier(ark_)


In [ ]:
# learning about how you can use "/" or fixed length identifiers that following a similar pattern -- for example, being minted by noid -- to impose structure
# checksums....
# can get a complicated mix -- so let me stick with "/" and look pathlike structures


In [ ]:
# how we're supposed to be sharing the test shoulder?
# ark:/99999/fk4{NAAN_and_shoulder}/

In [ ]:


import pathlib
p = pathlib.PurePath(ark_.postfix)
p.parts

In [ ]:
# create and delete reserved UUID in the TEST NAAN/TEST SHOULDER space

import uuid

id_ = str(uuid.uuid4().hex).replace("-", "")
arkid_ = sezid.ARKIdentifier(TEST_NAAN, TEST_SHOULDER, id_)



In [ ]:
%%pybash

ezid s "{EZID_USER}:{EZID_PASSWD}" delete "ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a6"

In [ ]:
%%pybash

ezid p "{EZID_USER}:{EZID_PASSWD}" view ark:/28722/k2154wc6r 

In [ ]:
%%pybash

curl -u "{EZID_USER}:{EZID_PASSWD}" -d format=anvl -d type=ark \
  -d permanence=real \
  -d createdAfter=2023-01-01T00:00:00Z \
  https://ezid.cdlib.org/download_request

In [ ]:
%%pybash

curl -u "{EZID_USER}:{EZID_PASSWD}" \
  -d format=csv \
  -d column=_id \
  -d column=erc.who \
  -d type=ark \
  -d permanence=real \
  -d createdAfter=2023-01-01T00:00:00Z \
  https://ezid.cdlib.org/download_request


In [ ]:
%%pybash

curl -u "{EZID_USER}:{EZID_PASSWD}" \
  -d format=csv \
  -d column=_id \
  -d column=erc.who \
  -d column=erc.what \
  -d column=erc.when \
  -d column=_owner \
  -d column=_ownergroup \
  -d column=_created \
  -d column=_updated \
  -d column=_profile \
  -d column=_target \
  -d column=_status \
  -d column=_export \
  -d type=ark \
  -d permanence=real \
  -d createdAfter=2023-04-01T00:00:00Z \
  https://ezid.cdlib.org/download_request



In [ ]:
import requests
url = 'https://ezid.cdlib.org/download_request'
data = {
    'format': 'csv',
    'column': ['_id', 'erc.who', 'erc.what', 'erc.when', '_owner', '_ownergroup', '_created', '_updated', '_profile', '_target', '_status', '_export'],
    'type': 'ark',
    'permanence': 'real',
    'createdAfter': '2023-04-01T00:00:00Z'
}

response = requests.post(url, auth=(EZID_USER, EZID_PASSWD), data=data)

if response.status_code == 200:
    print(response.text)
else:
    print(f"Error: {response.status_code}")

In [ ]:
response.status_code, response.text

In [ ]:
# write a public fake ARK

seems like we write arbitrary metadata fields. To delete the key, set it to ""


In [ ]:
%%pybash

ezid s "{EZID_USER}:{EZID_PASSWD}" create! ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a5 \
    _status public \
    erc.who "Raymond Yee" \
    erc.what "testing ark 00/05" \
    erc.when "(:unkn) Unknown time" \
    ry.who "" \
    profile erc

In [ ]:
from itertools import chain

client.args.server = "s"

metadata_ = {
  "ry.who": "Raymond Yee",
  "erc.who": "Nobody!",
  "profile": "erc",
  "erc.what": "testing ark 00/05",
  "erc.when": "(:unkn) Unknown time"
}


client.args.operation = ['create!', f'ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a5'] + list(chain(*metadata_.items()))
r = client.operation()
r

In [ ]:
# testing prefix matching



In [ ]:
%%pybash

ezid s "{EZID_USER}:{EZID_PASSWD}" view ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a5


In [ ]:
%%pybash

ezid -h

# arks structures to create

Things to figure out: theoretical and practical limits to ark sizes

In [ ]:
n = 10

'9' * 2**n

In [ ]:
# test of theoretical and practical limits


client = ect.Client()
client.args.credentials = f"{EZID_USER}:{EZID_PASSWD}"
client.args.server = "s"

postfix = '9' * 2**n




# Reserved IDs: creating and deleting

In [ ]:
%%pybash

# test shoulder and reserved id
# _status = reserved

ezid s "{EZID_USER}:{EZID_PASSWD}" create! "ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a7" \
    _status reserved \
    erc.who "Anthony Tuck" \
    erc.what "Photo 19660118 from Italy/Poggio Civitate/Tesoro/Tesoro 2F/1966, ID:476/PC 19660118" \
    erc.when "2017-10-04" \
    profile erc

In [ ]:
%%pybash

ezid s "{EZID_USER}:{EZID_PASSWD}" view "ark:/{TEST_NAAN}/{TEST_SHOULDER}/00/a7"

# Preregistration shoulder

In [ ]:
# issue of allowable characters

from urllib.parse import quote, unquote
import re


quote("A.B")

test_string = '12hello.there'
re.match(r'^[\w\-_.~]*$', test_string)

def uri_unreserved(s):
    """
    
    
    """
    return re.match(r'^[\w\-_.~]*$', s) is not None



quote("我") == '%E6%88%91'

uri_unreserved(test_string)



# Pathlib 

How well can we use the pathlib to model the "parts" of the ARK?


In [ ]:
import pathlib
import os



In [ ]:
p = pathlib.PurePath("a/b/c")
p

In [ ]:
p.name, p.parent, p.parent.parent, p.parent.parent.parent, p.parent.parent.parent.parent

In [ ]:
p1 = p / "d.py"
p1.suffix, p1.stem, p1.with_name('d1.py')


In [ ]:
"/".join(p.parts)

In [ ]:
p.match('b*/c')

In [ ]:
pathlib.PurePath("a/b/c").joinpath('../e') == pathlib.PurePath("a/b/e")

In [ ]:
p.parent / "e"

In [ ]:
pathlib.PurePath('a/.pyen/b/')

# Analyzing OpenContext arks

In [ ]:
import pandas as pd
from pandas import DataFrame, Series, Index
import numpy as np

In [ ]:
df = pd.read_csv("data/5lrRHNLMtGqBVN8d.csv")
df.head(2)

In [ ]:
len(df)

In [ ]:
df['_status'].value_counts()